In [13]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV # поиск по сетке

In [2]:
df = pd.read_csv('Advertising.csv')
df

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9
...,...,...,...,...
195,38.2,3.7,13.8,7.6
196,94.2,4.9,8.1,9.7
197,177.0,9.3,6.4,12.8
198,283.6,42.0,66.2,25.5


In [3]:
X = df.drop('sales', axis=1)
y = df['sales']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [6]:
scaler = StandardScaler()
scaler.fit(X_train) # обучаем только на обучающих данных, чтобы не было утечки данных

StandardScaler()

In [7]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
# ElasticNet - учитывает две регуляризации: Lasso и Ridge
# атрибуты класса:
# l1_ratio - используется для контроля соотношения между штрафами Lasso и Ridge регрессий
# l1_ratio = 0 - только Ridge 
# l1_ration = 1 - только Lasso
# параметр alpha - тоже является гиперпараметром
# по сути, необходимо найти комбинацию alpha и l1_ratio, чтобы они были оптимальными
# можно было с помощью циклов искать оптимальную комбинацию, но класс GridSearchCV позволит решить эту задачу автоматически.

In [ ]:
base_elastic_net_model = ElasticNet()

In [15]:
param_grid = {'alpha':[0.1,1,5,10,50,100],
              'l1_ratio':[.1, .5, .7, .9, .95, .99, 1]} # параметры сетки

In [ ]:
# estimator - обучающая модель
# param_grid - параметры сетки
# scoring - метрика
# cv - насколько частей разбивать данные при кросс-валидации
# verbose - нужно ли выводить детальную информации при расчёте. Удобно, когда алгоритм работает длительное время
grid_model = GridSearchCV(estimator=base_elastic_net_model, 
                          param_grid=param_grid, 
                          scoring='neg_mean_squared_error',
                          cv = 5, 
                          verbose=2) # создали модель

In [ ]:
# Теперь занимаемся её обучением
grid_model.fit(X_train, y_train)
# .fit() делает три вещи. 
# 1) Пройдет в цикле по всем гиперпараметрам
# 2) Кросс-валидация на k частей
# 3) Наилучшие параметры при указанной метрике


Fitting 5 folds for each of 42 candidates, totalling 210 fits
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.7; total time=   0.0s
[CV] END ............................alpha=0.1,

GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [0.1, 1, 5, 10, 50, 100],
                         'l1_ratio': [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]},
             scoring='neg_mean_squared_error', verbose=2)

In [ ]:
grid_model.best_estimator_ # наилучшая комбинация гиперпараметров

ElasticNet(alpha=0.1, l1_ratio=1)

In [20]:
grid_model.best_params_ # тоже самое, что и best_estimator_, но со словарём

{'alpha': 0.1, 'l1_ratio': 1}

In [22]:
# подробные результаты фитирования
grid_model.cv_results_

{'mean_fit_time': array([0.0032701 , 0.00064807, 0.00088935, 0.        , 0.00060859,
        0.00035996, 0.00077243, 0.        , 0.        , 0.00063634,
        0.00099168, 0.00061889, 0.00067129, 0.00086546, 0.00020003,
        0.00063987, 0.00062065, 0.00035872, 0.00082603, 0.00044208,
        0.00032973, 0.00053716, 0.00040007, 0.00050125, 0.00034366,
        0.00070863, 0.00030098, 0.00030198, 0.        , 0.        ,
        0.0003243 , 0.00080671, 0.        , 0.00033054, 0.00033846,
        0.        , 0.0008215 , 0.00066142, 0.00092487, 0.00131979,
        0.00021081, 0.00083132]),
 'std_fit_time': array([0.00478037, 0.00064933, 0.00077001, 0.        , 0.00049692,
        0.00071993, 0.00068787, 0.        , 0.        , 0.00077938,
        0.00053161, 0.00050561, 0.00053776, 0.00028104, 0.00040007,
        0.00061155, 0.00083236, 0.00071745, 0.00041368, 0.00065045,
        0.00065947, 0.00069211, 0.00048998, 0.00044756, 0.00068731,
        0.00060528, 0.00060196, 0.00060396, 0.   

In [24]:
# получаем таблицу с различными метриками и временем работы
#pd.DataFrame(grid_model.cv_results_)

In [25]:
y_pred = grid_model.predict(X_test)

In [26]:
mean_squared_error(y_test,y_pred)

2.387342642087474